# Memory

In [1]:
from secret_key import hugging_facehub_key
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hugging_facehub_key

In [72]:
from langchain import HuggingFaceHub
# Define LLM
repo_id = "google/flan-t5-large"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)

# Chat Message History

In [43]:
from langchain.memory import ChatMessageHistory
from langchain import PromptTemplate, LLMChain

In [44]:
# Initialize chat message history
history = ChatMessageHistory()

# Add user and AI messages to the history
history.add_user_message("hi!")
history.add_ai_message("whats up?")

In [45]:
# Print the current messages in the history
print(history.messages)

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]


In [46]:
# Add another user message and print the updated history
history.add_user_message("Fine, what about you?")
print(history.messages)

[HumanMessage(content='hi!'), AIMessage(content='whats up?'), HumanMessage(content='Fine, what about you?')]


In [47]:
# Define a prompt template with a question variable
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

In [48]:
# Create an LLMChain with the prompt template and a language model
# Note: 'llm' should be initialized before using it here
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [49]:
# Define a question and run the chain to get the model's response
question = "Which city does the company's headquarters for our international employees reside in?"
print(llm_chain.run(question))

The company's headquarters for our international employees reside in New York City. New York City is a city. New York City is a city. So the answer is New York.


## Conversation Buffer Memory

In [12]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [50]:
conversation.predict(input="Hi, my name is SAM")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: You have a day off tomorrow.
Human: What would be a good demo to show?
AI: I would like to show you the latest demo of the latest LLM demo.
Human: Hi, my name is SAM
AI:

> Finished chain.


'I am a system.'

In [14]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is tamarai selvan
AI: I am tamarai selvan
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is a number'

In [15]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is tamarai selvan
AI: I am tamarai selvan
Human: What is 1+1?
AI: 1+1 is a number
Human: What is my name?
AI:

> Finished chain.


'tamarai selvan'

In [16]:
print(memory.buffer)

Human: Hi, my name is tamarai selvan
AI: I am tamarai selvan
Human: What is 1+1?
AI: 1+1 is a number
Human: What is my name?
AI: tamarai selvan


In [17]:
memory.load_memory_variables({})

{'history': 'Human: Hi, my name is tamarai selvan\nAI: I am tamarai selvan\nHuman: What is 1+1?\nAI: 1+1 is a number\nHuman: What is my name?\nAI: tamarai selvan'}

In [18]:
memory = ConversationBufferMemory()

In [19]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [20]:
print(memory.buffer)

Human: Hi
AI: What's up


In [21]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [22]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [23]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

## ConversationBufferWindowMemory

In [24]:
from langchain.memory import ConversationBufferWindowMemory

In [25]:
memory = ConversationBufferWindowMemory(k=1)               

In [26]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [27]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [28]:
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [29]:
conversation.predict(input="Hi, my name is Andrew")

'Andrew: Hello, how can I help you?'

In [30]:
conversation.predict(input="What is 1+1?")

'Andrew: 1+1 is a number that is a combination of two numbers.'

In [31]:
conversation.predict(input="What is my name?")

'Andrew: Andrew is a fictional character.'

## ConversationTokenBufferMemory

In [32]:
#!pip install tiktoken

In [33]:
from langchain.memory import ConversationTokenBufferMemory

In [34]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

C:\Users\Dell\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [35]:
memory.load_memory_variables({})

{'history': 'Human: AI is what?!\nAI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

In [36]:
from langchain.memory import ConversationSummaryBufferMemory

In [37]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [38]:
memory.load_memory_variables({})

{'history': 'System: AI is hanging out.\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}

In [39]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [40]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: AI is hanging out.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'I would like to show you the latest demo of the latest LLM demo.'

In [41]:
memory.load_memory_variables({})

{'history': 'System: You have a day off tomorrow.\nHuman: What would be a good demo to show?\nAI: I would like to show you the latest demo of the latest LLM demo.'}

## Knowledge Graph Memory

In [115]:
from langchain.memory import ConversationKGMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain import HuggingFaceHub
import networkx as nx
import matplotlib.pyplot as plt

In [116]:
memory = ConversationBufferWindowMemory(k=1)  

In [121]:
repo_id = "google/flan-t5-large"  # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)

In [125]:
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

In [123]:
conversation_with_kg = ConversationChain(
    llm=llm, 
    verbose=True, 
    prompt=prompt,
    memory=ConversationKGMemory(llm=llm)
)

In [124]:
conversation_with_kg.predict(input="Hi there! I am Sam")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Hi there! I am Sam
AI:

> Finished chain.


'Sam'

In [126]:
conversation_with_kg.predict(input="My TV is broken and I need some customer assistance")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: My TV is broken and I need some customer assistance
AI:

> Finished chain.


'I can help you with that. What kind of TV is it?'

In [127]:
conversation_with_kg.predict(input="My TV is LED sunsung")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: My TV is LED sunsung
AI:

> Finished chain.


'yes'

In [128]:
conversation_with_kg.predict(input="Yes it is and it is still under warranty. my warranty number is A512423")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Yes it is and it is still under warranty. my warranty number is A512423
AI:

> Finished chain.


'Is it still under warranty?'

In [129]:
# nx.draw(conversation_with_kg.memory.kg, with_labels=True)
# plt.show()

print(conversation_with_kg.memory.kg)
print(conversation_with_kg.memory.kg.get_triples())

[('Sam)', '"Sam', 'PERSON'), ('human)', 'LED Sunsun', 'TV_TYPE')]
